In [13]:
%run ./function.ipynb

# Load data

In [2]:
#PATH = 'C:/Users/NCTUUser2/Desktop/mark/'
PATH  = 'C:/Users/mark0/Desktop/捷智/MetaEdge/'
FILE = 'TWN_A11_01_day15_with_aggregated_txn_scenario_txn_type_key_營業日.csv'
df = pd.read_csv(PATH + FILE)
df.columns

Index(['alert_id', 'run_date', 'primary_entity_level_code',
       'primary_entity_number', 'scenario_name', 'TO_SAR', 'Acct_No',
       'Cust_No', 'Cust_Key', 'Cust_Segmentation', 'Cust_Typ_Cd', 'Cust_Id_Id',
       'Cust_Id_Typ_Desc', 'Cust_Id_State_Cd', 'Cust_Dt_Of_Birth',
       'Cust_Sts_Desc', 'Street_Postal_Cd', 'Street_Cntry_Cd',
       'Mailing_Postal_Cd', 'Mailing_Cntry_Cd', 'Residence_Cntry_Cd',
       'Citizenship_Cntry_Cd', 'Org_Cntry_Of_Buz_Cd', 'Empl_No', 'Ocup_Code',
       'job_title_code', 'Lst_Cont_Dt', 'Politically_Exposed_Pers_Ind',
       'Non_Prft_Org_Ind', 'Cust_Since_Dt', 'Lst_Susp_Actv_Rpt_Dt',
       'Extnl_Cust_Ind', 'Negative_News_Ind', 'Prim_Br_No', 'Obu_Ind',
       'Riskclass_Cd', 'risk_score', 'last_risk_assessment_date',
       'change_current_ind', 'change_begin_date', 'change_end_date',
       'Credit_Amt', 'number_of_Credit', 'Debit_Amt', 'number_of_Debit'],
      dtype='object')

In [3]:
df['TO_SAR'] = df['TO_SAR'].replace(np.nan, 0)
df['TO_SAR'] = df['TO_SAR'].replace('F', 1)
df['TO_SAR'] = df['TO_SAR'].replace('NF', 1)

In [5]:
SAR_type = 'TO_SAR'
train_set, test_set = ReadFile(df, SAR_type)

Verify on time series
shape of data: (86770, 39)
total number of TO_SAR: 446
********************************
time interval in train set:2019-03-04~2020-04-29
TO_SAR in train set: 267
TO_SAR/Total in train set 0.005128599143312653
split dataset to train, test...
********************************
time interval in test set:2020-02-19~2020-12-31
TO_SAR in test set: 179
TO_SAR/Total in test set 0.0051571638479933155
********************************
train data shape:  (52061, 38)
test data shape:  (34709, 38)


C:\Users\mark0\AppData\Local\Temp\ipykernel_6316\2736333543.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train = train_Non_SAR.append(train_TO_SAR)
C:\Users\mark0\AppData\Local\Temp\ipykernel_6316\2736333543.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test = test_Non_SAR.append(test_TO_SAR)


In [7]:
train_set = train_set.dropna(subset= ['Credit_Amt', 'number_of_Credit', 'Debit_Amt', 'number_of_Debit']).reset_index(drop= True)
test_set = test_set.dropna(subset= ['Credit_Amt', 'number_of_Credit', 'Debit_Amt', 'number_of_Debit']).reset_index(drop= True)
print(f'train_set:{train_set.shape}\n' + f'test_set:{test_set.shape}')

train_set:(52060, 38)
test_set:(34709, 38)


In [18]:
SAR = train_set[train_set.TO_SAR!= 0]
Non_SAR = train_set[train_set.TO_SAR== 0]

# Full dimemsion newton method

In [19]:
feature_name = ['Credit_Amt', 'number_of_Credit', 'Debit_Amt', 'number_of_Debit']
K = len(feature_name)

In [20]:
kdtree = cKDTree(train_set[feature_name], leafsize= 100)
tmp = (SAR.loc[:, 'Credit_Amt'].to_numpy()**2 + SAR.loc[:, 'number_of_Credit'].to_numpy()**2 \
    + SAR.loc[:, 'Debit_Amt'].to_numpy()**2 + SAR.loc[:, 'number_of_Debit'].to_numpy()**2) ** (1/2)

SAR.insert(0, 'distance', tmp)
del tmp

In [22]:
idx_distance = SAR.loc[SAR['distance'] != 0].sort_values(by= 'distance').index

In [23]:
distance_results = pd.DataFrame(columns= [feature_name[0], feature_name[1], feature_name[2], feature_name[3], 'recall', 'filterRate', 'distance'])

for i, idx in enumerate(idx_distance):
    point = SAR.loc[idx, feature_name]
    distance = np.sum(point**2) ** (1/2)
    recall_tmp, filterRate_tmp = RecallFilterRate(point, feature_name, train_set)  
    tmp = pd.DataFrame({feature_name[0]: point[0], feature_name[1]: point[1], feature_name[2]: point[2], feature_name[3]: point[3], 'recall': recall_tmp, 'filterRate': filterRate_tmp, 'distance': distance}, index= [i])
    distance_results = pd.concat([distance_results, tmp])

In [24]:
distance_results.sort_values(by= ['distance', 'recall'], ascending= True)

,Credit_Amt,number_of_Credit,Debit_Amt,number_of_Debit,recall,filterRate,distance
0,0.0,0.0,2462.0,2.0,0.812734,0.362063,2.462001e+03
1,200920.0,2.0,0.0,0.0,0.340824,0.445294,2.009200e+05
2,0.0,0.0,270000.0,1.0,0.790262,0.428333,2.700000e+05
3,0.0,0.0,300000.0,6.0,0.730337,0.529677,3.000000e+05
4,0.0,0.0,549945.0,14.0,0.516854,0.756473,5.499450e+05
...,...,...,...,...,...,...,...
262,6840000.0,14.0,0.0,0.0,0.011236,0.994602,6.840000e+06
263,0.0,0.0,7590025.0,8.0,0.011236,0.994103,7.590025e+06
264,0.0,0.0,10160040.0,9.0,0.007491,0.998617,1.016004e+07
265,0.0,0.0,10550015.0,7.0,0.007491,0.998483,1.055002e+07


In [25]:
point0 = distance_results.sort_values(by= 'recall', ascending= False).iloc[0, 0:K]
point0 = point0.to_numpy().reshape(K,1)

In [37]:
# loss finction
def loss_fn(point, IG, recall):
    #lowerbound = np.sum(np.log(point))
    #upperbound = 0
    # for x, Mx in zip(point, Max_X):
    #     upperbound += np.log(MX- x)
    loss = -IG
    #loss = IG + lowerbound + upperbound
    #loss = IG + lowerbound + upperbound
    #loss = IG + lowerbound + upperbound + recall
    #loss = IG * sigmoid(recall) + lowerbound + upperbound
    return loss

In [38]:
def GradIGUpdate(point, feature_name, data):
    n = 0
    results = pd.DataFrame(columns= ['iteration number', feature_name[0], feature_name[1], feature_name[2], feature_name[3], \
                    'new_'+feature_name[0], 'new_'+feature_name[1], 'new_'+feature_name[2], 'new_'+feature_name[3], \
                        'recall', 'filter rate', 'loss'])

    information_gain = -informationGain(point, feature_name, data)
    recall, filter_rate = RecallFilterRate(point, feature_name, data)
    # loss = -informtion_gain 
    loss = loss_fn(point, -information_gain, recall)

    tmp_results = pd.DataFrame({'iteration number':n, feature_name[0]: float(point[0]), feature_name[1]: float(point[1]), feature_name[2]: float(point[2]), feature_name[3]: float(point[3]) \
                    , 'new_'+feature_name[0]: float(point[0]), 'new_'+feature_name[1]: float(point[1]), 'new_'+feature_name[2]: float(point[2]), 'new_'+feature_name[3]: float(point[3]) \
                    ,'information gain': -information_gain, 'recall': recall, 'filter rate': filter_rate, 'loss': loss}, index= [n])
    results = pd.concat([results, tmp_results])
    n += 1

    while True:
        if n > N_max:
            break
        
        gradIG = gradientIG(point, feature_name, data)
        #gradIG = gradIG / np.linalg.norm(gradIG)
        print('gradIG:\n', gradIG)
        if np.all(gradIG == 0):
            print('gradIG has zero')
        new_point = point - step_size * gradIG

        information_gain = -informationGain(new_point, feature_name, data)
        recall, filter_rate = RecallFilterRate(new_point, feature_name, data)
        # loss = -information_gain 
        loss = loss_fn(point, information_gain, recall)

        print(f'n={n}, {feature_name[0]}:{float(new_point[0])}, {feature_name[1]}: {float(new_point[1])}' \
            ,f'{feature_name[2]}: {float(new_point[2])}, {feature_name[3]}: {float(new_point[3])}' \
            , f'\ninformation gain: {information_gain}, recall:{recall}, filter rate:{filter_rate}, loss: {loss}')
        
        tmp_results = pd.DataFrame({'iteration number':n, feature_name[0]: float(point[0]), feature_name[1]: float(point[1]), feature_name[2]: float(point[2]), feature_name[3]: float(point[3]) \
                        , 'new_'+feature_name[0]: float(new_point[0]), 'new_'+feature_name[1]: float(new_point[1]), 'new_'+feature_name[2]: float(new_point[2]), 'new_'+feature_name[3]: float(new_point[3]) \
                        ,'information gain': information_gain, 'recall': recall, 'filter rate': filter_rate, 'loss': loss}, index= [n])

        results = pd.concat([results, tmp_results])
        n += 1
        point = new_point

    return results


In [32]:
#hyperparameter
SAR_type = 'TO_SAR'
step_size = 1
N_max = 1000
father_entropy = Entropy(train_set, SAR_type)

In [39]:
# loss funciton only information gain 
if __name__ == '__main__':
    np.random.seed(1111)
    print('initial point\n', point0)
    results = GradIGUpdate(point0, feature_name, train_set)

initial point
 [[0.000e+00]
 [0.000e+00]
 [2.462e+03]
 [2.000e+00]]
gradIG:
 [[-2.64009268e-09]
 [-1.47400868e-05]
 [ 1.66750899e-09]
 [-1.62843352e-05]]
n=1, Credit_Amt:2.6400926807312637e-09, number_of_Credit: 1.4740086815880071e-05 Debit_Amt: 2461.9999999983324, number_of_Debit: 2.0000162843352336 
information gain: -1.6642798439450712e-05, recall:0.2209737827715356, filter rate:0.7504610065309258, loss: 1.6642798439450712e-05
gradIG:
 [[-0.]
 [-0.]
 [ 0.]
 [-0.]]
gradIG has zero
n=2, Credit_Amt:2.6400926807312637e-09, number_of_Credit: 1.4740086815880071e-05 Debit_Amt: 2461.9999999983324, number_of_Debit: 2.0000162843352336 
information gain: -1.6642798439450712e-05, recall:0.2209737827715356, filter rate:0.7504610065309258, loss: 1.6642798439450712e-05
gradIG:
 [[-0.]
 [-0.]
 [ 0.]
 [-0.]]
gradIG has zero
n=3, Credit_Amt:2.6400926807312637e-09, number_of_Credit: 1.4740086815880071e-05 Debit_Amt: 2461.9999999983324, number_of_Debit: 2.0000162843352336 
information gain: -1.66427984

In [ ]:
results.loc[[0, 1000]]

In [ ]:
plot_results(results)

In [ ]:
plot_seperator(results)